In [1]:
#!/usr/bin/env python

# Stage-IV-at-COOP_PR.ipynb

In [2]:
''' 
   Stage-IV-at-COOP_PR.ipynb

   Read in stage-IV houlry precipitation at the location of COOP stations
   by using inverse distance averaging of the four closest gridcells and the closest grid cell
   
   This program needs data from:
   papers/2021_Hist-Ext-PR-Changes/programs/COOP_Station_preprocessor/COOP_Station_preprocessor.ipynb
   
'''

' \n   Stage-IV-at-COOP_PR.ipynb\n\n   Read in stage-IV houlry precipitation at the location of COOP stations\n   by using inverse distance averaging of the four closest gridcells and the closest grid cell\n   \n   This program needs data from:\n   papers/2021_Hist-Ext-PR-Changes/programs/COOP_Station_preprocessor/COOP_Station_preprocessor.ipynb\n   \n'

In [3]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import glob
import os
from pdb import set_trace as stop
from scipy.ndimage.filters import gaussian_filter
from scipy.ndimage import median_filter
from scipy.ndimage import label
from matplotlib import cm
from scipy import ndimage
import random
import scipy
import pickle
import datetime
import pandas as pd
import subprocess
from calendar import monthrange
import pandas as pd
import datetime
import sys 
import shapefile as shp
import matplotlib.path as mplPath
from scipy.stats import norm
import matplotlib.gridspec as gridspec
# from mpl_toolkits.basemap import Basemap, cm
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.gridspec as gridspec
from pylab import *
import string
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
import shapefile
from calendar import monthrange
from tqdm import tqdm


# # fix pickle load issue
# np_load_old = np.load
# # modify the default parameters of np.load
# np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)



# from: https://rafatieppo.github.io/post/2018_07_27_idw2pyr/
# packages
import math
import numpy as np
#------------------------------------------------------------
# Distance calculation, degree to km (Haversine method)
def harvesine(lon1, lat1, lon2, lat2):
    rad = math.pi / 180  # degree to radian
    R = 6378.1  # earth average radius at equador (km)
    dlon = (lon2 - lon1) * rad
    dlat = (lat2 - lat1) * rad

    a = (np.sin(dlat / 2)) ** 2 + np.cos(lat1 * rad) * \
        np.cos(lat2 * rad) * (np.sin(dlon / 2)) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    d = R * c
    return(d)

# ------------------------------------------------------------
# Prediction
def idwr(x, y, z, xi, yi):
    lstxyzi = []
    for p in range(len(xi)):
        lstdist = []
        for s in range(len(x)):
            d = (harvesine(x[s], y[s], xi[p], yi[p]))
            lstdist.append(d)
        sumsup = list((1 / np.power(lstdist, 2)))
        suminf = np.sum(sumsup)
        sumsup = np.sum(np.array(sumsup) * np.array(z))
        u = sumsup / suminf
        xyzi = [xi[p], yi[p], u]
        lstxyzi.append(xyzi)
    return(lstxyzi)

###  USER MODIFY SECTION

In [4]:
StartDay = datetime.datetime(2001, 10, 1,0)
StopDay = datetime.datetime(2018, 12, 31,23)
rgdTimeFULL=pd.date_range(StartDay, end=StopDay, freq='h')
rgdTimeFULLDD=pd.date_range(StartDay, end=StopDay, freq='d')
rgdTimeFULLMM=pd.date_range(StartDay, end=StopDay, freq='m')
Years = np.unique(rgdTimeFULL.year)

SaveDir = '/glade/campaign/mmm/c3we/prein/Papers/2021_Hist-Ext-PR-Changes/data/'

### Read STAGE-IV coordinates

In [5]:
ncid=Dataset('/glade/campaign/mmm/c3we/prein/observations/STAGE_II_and_IV/data/LEVEL_2-4_hourly_precipitation_200202.nc', mode='r') # open the netcdf
Lon4=np.squeeze(ncid.variables['lon'][:])
Lat4=np.squeeze(ncid.variables['lat'][:])
ncid.close()


### Read in COOP locations

In [6]:
# This data comes from - papers/2021_Hist-Ext-PR-Changes/programs/COOP_Station_preprocessor/COOP_Station_preprocessor.ipynb

COOPsave = '/glade/campaign/mmm/c3we/prein/Papers/2021_Hist-Ext-PR-Changes/data/CCOP_stations_1979-2020.npz'
DATA = np.load(COOPsave, allow_pickle=True)
LonSTCO=DATA['LonSTCO']
LatSTCO=DATA['LatSTCO']
AltSTCO=DATA['AltSTCO']
RatioMissing=DATA['RatioMissing']

### Get the four closest grid cells and their distances to each station

In [7]:
Lat1D = np.array(np.reshape(Lat4, Lat4.shape[0]*Lat4.shape[1]))
Lon1D = np.array(np.reshape(Lon4, Lon4.shape[0]*Lon4.shape[1]))

N_closest = 4

Distance = np.zeros((len(LonSTCO),N_closest)); Distance[:] = np.nan
GC_ID_closest = np.zeros((len(LonSTCO),2,N_closest)); GC_ID_closest[:] = np.nan
for st in tqdm(range(len(LonSTCO))):
    DIST = harvesine(Lon1D, Lat1D,LonSTCO[st], LatSTCO[st])
    Closest4 = np.argsort(DIST)[:N_closest]
    if np.min(DIST) < 4:
        Distance[st,:] = DIST[Closest4]
        for ii in range(N_closest):
            GC_ID_closest[st,:,ii] = np.unravel_index(Closest4[ii].astype(int), (Lat4.shape[0],Lat4.shape[1]))

100%|██████████| 1983/1983 [06:45<00:00,  4.89it/s]


### Read in Stage-IV precipitation and interpolate to station locations using inverse discance averaging

In [7]:
# WRF_stationPR = np.zeros((len(rgdTimeFULL),len(LonSTCO))); WRF_stationPR[:] = np.nan


# # Read data month by month and save each month to speed up the processing
# for mm in tqdm(range(0, len(rgdTimeFULLMM),1)):
#     iTIME = (rgdTimeFULLMM[mm].month == rgdTimeFULL.month) & (rgdTimeFULLMM[mm].year == rgdTimeFULL.year)
#     sTime = str(rgdTimeFULLMM[mm].year)+str(rgdTimeFULLMM[mm].month).zfill(2)
#     TMP_file = '/glade/campaign/mmm/c3we/prein/Papers/2021_Hist-Ext-PR-Changes/data/WRF/WRF-PR-at-COOP_'+sTime+'.npz'
#     if os.path.exists(TMP_file) == False:

#         ObservedData='/glade/campaign/mmm/c3we/prein/CONUS404/data/MonthlyData/PREC_ACC_NC_'+sTime+'_CONUS404.nc'
#         ncid=Dataset(ObservedData, mode='r')
#         DATA = np.squeeze(ncid.variables['PREC_ACC_NC'][:,:,:])
#         ncid.close()
# #         DATA = np.reshape(DATA, (DATA.shape[0],DATA.shape[1]*DATA.shape[2]))
#         for st in range(len(LonSTCO)):
#             if np.min(Distance[st,:]) < 4:
#                 PRact = DATA[:,GC_ID_closest[st,0,:].astype('int'),GC_ID_closest[st,1,:].astype('int')]
#                 DistanceAct = Distance[st,:]**-1
#                 WRF_stationPR[iTIME,st] = np.sum(PRact*DistanceAct[None,:], axis=1)/np.sum(DistanceAct)
#         np.savez(TMP_file,
#                  time=rgdTimeFULL[iTIME],
#                  WRF_stationPR=WRF_stationPR[iTIME,:],
#                  LonSTCO=LonSTCO,
#                  LatSTCO=LatSTCO)
#     else:
#         DATA = np.load(TMP_file)
#         WRF_stationPR[iTIME,:]=DATA['WRF_stationPR']
        

 83%|████████▎ | 408/492 [04:13<00:52,  1.61it/s]


FileNotFoundError: [Errno 2] No such file or directory: b'/glade/campaign/mmm/c3we/prein/CONUS404/data/MonthlyData/PREC_ACC_NC_201310_CONUS404.nc'

### USE THE CLOSEST GRIDCELL FOR STATION EVALUATION 

In [8]:
stage_IV_stationPR = np.zeros((len(rgdTimeFULL),len(LonSTCO))); stage_IV_stationPR[:] = np.nan


# Read data month by month and save each month to speed up the processing
for mm in tqdm(range(0, len(rgdTimeFULLMM),1)):
    iTIME = (rgdTimeFULLMM[mm].month == rgdTimeFULL.month) & (rgdTimeFULLMM[mm].year == rgdTimeFULL.year)
    sTime = str(rgdTimeFULLMM[mm].year)+str(rgdTimeFULLMM[mm].month).zfill(2)
    TMP_file = '/glade/campaign/mmm/c3we/prein/Papers/2021_Hist-Ext-PR-Changes/data/stage-IV/stage-IV-PR-at-COOP_'+sTime+'_closest-GC.npz'
    if os.path.exists(TMP_file) == False:

        ObservedData='/glade/campaign/mmm/c3we/prein/observations/STAGE_II_and_IV/data/LEVEL_2-4_hourly_precipitation_'+sTime+'.nc'
        ncid=Dataset(ObservedData, mode='r')
        DATA = np.squeeze(ncid.variables['Precipitation'][:,:,:])
        ncid.close()
#         DATA = np.reshape(DATA, (DATA.shape[0],DATA.shape[1]*DATA.shape[2]))
#         for st in range(len(LonSTCO)):
#             if np.min(Distance[st,:]) < 4:
        NAN = np.isnan(GC_ID_closest[:,0,0])
        Indices = GC_ID_closest[:,:,0].astype('int'); Indices[NAN] = 0
        
        stage_IV_stationPR[iTIME,:] = DATA[:,Indices[:,0],Indices[:,1]]
        PR = stage_IV_stationPR[iTIME,:]; PR[:,NAN] = np.nan
        np.savez(TMP_file,
                 time=rgdTimeFULL[iTIME],
                 stage_IV_stationPR=PR,
                 LonSTCO=LonSTCO,
                 LatSTCO=LatSTCO)
    else:
        DATA = np.load(TMP_file)
        stage_IV_stationPR[iTIME,:]=DATA['stage_IV_stationPR']
        

100%|██████████| 207/207 [1:05:59<00:00, 19.13s/it]


In [10]:
DATA[:,GC_ID_closest[:,0,0].astype('int'),GC_ID_closest[:,1,0].astype('int')].shape

IndexError: index -9223372036854775808 is out of bounds for axis 1 with size 1015

In [11]:
DATA.shape

(720, 1015, 1367)

In [13]:
GC_ID_closest[:,0,0]

array([321., 389., 333., ...,  nan,  nan,  nan])